In [7]:
import os
import importlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp

from utils import read_intan as ri
from utils import signal_preprocessing as spp
from utils import plot_chuncked_data as pcd

In [8]:
raw_path = "/Volumes/CHOO'S SSD/LINK/eeg/s-eeg/d21"
# npz_path = './data/compressed/attempt1'
exp_name = raw_path.split('/')[-1]
pin_map = {
        1: 'in11', 2: 'in10', 3: 'in9', 4: 'in8',
        5: 'in23', 6: 'in22', 7: 'in21', 8: 'in20',
        9: 'in12', 10: 'in13', 11: 'in14', 12: 'in15',
        13: 'in16', 14: 'in17', 15: 'in18', 16: 'in19'
    }

In [9]:
raw_df, metadata = ri.rhd2dataframe(raw_path, pin_map)

raw_df.head(10)

Found 29 RHD files in /Volumes/CHOO'S SSD/LINK/eeg/s-eeg/d21
Successfully loaded data from 250630 pilo_250630_155325.rhd
Successfully loaded data from 250630 pilo_250630_155825.rhd
Successfully loaded data from 250630 pilo_250630_160325.rhd
Successfully loaded data from 250630 pilo_250630_160825.rhd
Successfully loaded data from 250630 pilo_250630_161325.rhd
Successfully loaded data from 250630 pilo_250630_161825.rhd
Successfully loaded data from 250630 pilo_250630_162325.rhd
Successfully loaded data from 250630 pilo_250630_162825.rhd
Successfully loaded data from 250630 pilo_250630_163325.rhd
Successfully loaded data from 250630 pilo_250630_163825.rhd
Successfully loaded data from 250630 pilo_250630_164325.rhd
Successfully loaded data from 250630 pilo_250630_164825.rhd
Successfully loaded data from 250630 pilo_250630_165325.rhd
Successfully loaded data from 250630 pilo_250630_165825.rhd
Successfully loaded data from 250630 pilo_250630_170325.rhd
Successfully loaded data from 250630 pi

,time,pin_4,pin_3,pin_2,pin_1,pin_9,pin_7,pin_6,pin_5
0,0.00000,-15.405,-14.040,45.630,-18.720,357.240,-74.880,-7.995,-54.210
1,0.00005,-19.890,-14.430,45.630,-21.450,348.855,-80.925,-8.775,-54.210
2,0.00010,-22.815,-23.985,45.630,-24.960,349.440,-80.925,-7.215,-54.600
3,0.00015,-19.110,-15.015,45.630,-21.255,357.240,-80.925,-1.755,-54.210
4,0.00020,-15.990,-11.505,45.630,-17.355,348.855,-81.900,-4.875,-45.825
5,0.00025,-11.895,6.825,51.870,-16.965,352.560,-75.075,-0.975,-52.065
6,0.00030,-11.895,6.240,61.035,-8.775,357.240,-75.855,7.410,-54.210
7,0.00035,-4.680,10.920,62.205,-8.775,357.240,-72.345,8.775,-41.145
8,0.00040,3.705,18.720,62.205,-8.775,358.800,-71.565,7.605,-39.585
9,0.00045,3.705,10.920,62.205,-7.020,363.480,-66.300,7.605,-39.585


In [10]:
df = raw_df.copy()

In [11]:
mosaic_groups = {
    'lat': [(4, 1), (1, 5), (5, 6), (6, 7)],
    'med': [(4, 2), (2, 3), (3, 9)],
    'ref_shift': [(4, 7), (1, 7), (2, 7), (5, 7), (3, 7), (6, 7), (9, 7)]
}

# df = spp.apply_common_average_reference(df)
# df = spp.zscore_normalize(df)
mosaic_data = spp.get_mosaic_df(df, mosaic_groups)

In [ ]:
mosaic_data['lat'].head(10)

,time,pin_4-pin_1,pin_1-pin_5,pin_5-pin_6,pin_6-pin_7
0,0.00000,3.315,35.490,-46.215,66.885
1,0.00005,1.560,32.760,-45.435,72.150
2,0.00010,2.145,29.640,-47.385,73.710
3,0.00015,2.145,32.955,-52.455,79.170
4,0.00020,1.365,28.470,-40.950,77.025
5,0.00025,5.070,35.100,-51.090,74.100
6,0.00030,-3.120,45.435,-61.620,83.265
7,0.00035,4.095,32.370,-49.920,81.120
8,0.00040,12.480,30.810,-47.190,79.170
9,0.00045,10.725,32.565,-47.190,73.905


: 

In [ ]:
def plot_channels(df, time_min=None, time_max=None):
    """Plot each channel in df as a subplot with symmetric y-limits based on nearest multiple of 50 and exact x-axis range."""
    # Filter by time range if specified
    df_plot = df
    if time_min is not None:
        df_plot = df_plot[df_plot['time'] >= time_min]
    if time_max is not None:
        df_plot = df_plot[df_plot['time'] <= time_max]
    # Identify channel columns
    channels = [col for col in df_plot.columns if col != 'time']
    # Compute global min/max across all channels
    data_vals = df_plot[channels].values
    global_max = data_vals.max()
    global_min = data_vals.min()
    abs_max = max(abs(global_max), abs(global_min))
    # Round up to nearest multiple of 50 for symmetric limits
    y_limit = int(np.ceil(abs_max / 50.0) * 50)
    # Create subplots
    n_ch = len(channels)
    fig, axs = plt.subplots(n_ch, 1, sharex=True, figsize=(10, 2*n_ch))
    if n_ch == 1:
        axs = [axs]
    for ax, ch in zip(axs, channels):
        ax.plot(df_plot['time'], df_plot[ch])
        ax.set_ylim(-y_limit, y_limit)
        ax.set_ylabel(ch)
    # Set exact x-axis limits and remove padding
    x_start = time_min if time_min is not None else df_plot['time'].min()
    x_end = time_max if time_max is not None else df_plot['time'].max()
    for ax in axs:
        ax.set_xlim(x_start, x_end)
        ax.margins(x=0)
    axs[-1].set_xlabel('time')
    plt.tight_layout()
    return fig, axs

start_time = 200
duration = 15  # Duration in seconds to plot
end_time = start_time + duration

for group in list(mosaic_data.keys()):
    mosaic_df = mosaic_data[group]
    # mosaic_df = spp.remove_artifacts_hilbert_kurtosis(mosaic_df)
    mosaic_df = spp.bandpass_filter(mosaic_df, 1, 100, order=4)
    # mosaic_df = spp.resample_dataframe(mosaic_df, 1000)
    fig, axs = plot_channels(mosaic_df, start_time, end_time)


Estimated sampling rate: 20000.0 Hz
